In [24]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torchvision import datasets, transforms
from datasets import load_dataset
from pytorch_grad_cam import run_dff_on_image,\
    GradCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional
#gpu
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA!")
else:
    device = torch.device("cpu")
    print("Using CPU!")


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

root_path = '/home/workstation/code/XAImethods/pytorch-grad-cam/ImageNet-Mini/images'  
dataset = datasets.ImageFolder(root=root_path, transform=transform)




""" Model wrapper to return a tensor"""
class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

""" Translate the category name to the category index.
    Some models aren't trained on Imagenet but on even larger datasets,
    so we can't just assume that 761 will always be remote-control.

"""
def category_name_to_index(model, category_name):
    name_to_index = dict((v, k) for k, v in model.config.id2label.items())
    return name_to_index[category_name]
    
""" Helper function to run GradCAM on an image and create a visualization.
    (note to myself: this is probably useful enough to move into the package)
    If several targets are passed in targets_for_gradcam,
    e.g different categories,
    a visualization for each of them will be created.
    
"""
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          reshape_transform: Optional[Callable],
                          input_tensor: torch.nn.Module=img_tensor,
                          input_image: Image=image,
                          method: Callable=GradCAM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                 target_layers=[target_layer],
                 reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image)/255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1]//2, visualization.shape[0]//2))
            results.append(visualization)
        return np.hstack(results)
    
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

Using CUDA!


Dataset

In [27]:
print(image.shape)
print(img_tensor.shape)


torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])


In [21]:
import pandas as pd
# Read CSV
csv_path = "/home/workstation/code/XAImethods/pytorch-grad-cam/img_name_label.csv"
df = pd.read_csv(csv_path)

# Create dataset
root_dir = "/home/workstation/code/XAImethods/pytorch-grad-cam/data"



# Custom Dataset
class CustomImageNetDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe.set_index(dataframe.columns[1])  # 将图片名设置为索引
        self.file_list = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_list[idx])
        image = Image.open(img_name)
        
        # 从表格中获取标签
        if self.file_list[idx] in self.dataframe.index:
            label = int(self.dataframe.loc[self.file_list[idx], self.dataframe.columns[2]])

        else:
            raise ValueError(f"Image {self.file_list[idx]} not found in the CSV.")

        if self.transform:
            image = self.transform(image)

        return image, label



image, label = testdataset[999]
img_tensor = transforms.ToTensor()(image)


NUM_IMAGES = 20  # 设置为您要处理的图像数量，例如20

for idx in range(NUM_IMAGES):
    image, label = testdataset[idx]
    img_tensor = transforms.ToTensor()(image).to(device)
    
    # 获取预测的类别和分数
    logits = model(img_tensor.unsqueeze(0)).logits
    probs = torch.nn.functional.softmax(logits, dim=1)
    top_probs, top_indices = torch.topk(probs, 5)
    top_probs = top_probs.cpu().detach().numpy().squeeze()
    top_indices = top_indices.cpu().detach().numpy().squeeze()
    
    print(f"Image {idx + 1}/{NUM_IMAGES}")
    print("-" * 40)
    
    for i in range(5):
        category = model.config.id2label[top_indices[i]]
        score = top_probs[i]
        print(f"Predicted class {i + 1}: {category} with score: {score:.4f}")
    
    # 运行GradCAM
    gradcam_result = run_grad_cam_on_image(model=model,
                                           target_layer=target_layer,
                                           targets_for_gradcam=targets_for_gradcam,
                                           reshape_transform=None,
                                           input_tensor=img_tensor,
                                           input_image=image)
    
    # 显示GradCAM结果
    display(Image.fromarray(gradcam_result))
    
    print("\n\n")


NameError: name 'Dataset' is not defined

Resnet

In [28]:
from transformers import ResNetForImageClassification
from PIL import Image
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")


# We will show GradCAM for the "Egyptian Cat" and the 'Remote Control" categories:
targets_for_gradcam = [ClassifierOutputTarget(category_name_to_index(model, "Egyptian cat")),
                       ClassifierOutputTarget(category_name_to_index(model, "remote control, remote"))]

# The last layer in the Resnet Encoder:
target_layer = model.resnet.encoder.stages[-1].layers[-1]

for image, label in dataset:
    img_tensor = image.unsqueeze(0).to(device)  # 增加一个批次维度并移到设备上
    
    # ... 这里运行您的run_dff_on_image, run_grad_cam_on_image等函数...
    # 例如:
    display(Image.fromarray(run_dff_on_image(model=model,
                                             target_layer=target_layer,
                                             classifier=model.classifier,
                                             img_pil=Image.fromarray(image.mul(255).byte().cpu().numpy().transpose(1, 2, 0)),
                                             img_tensor=img_tensor,
                                             reshape_transform=None,
                                             n_components=4,
                                             top_k=2)))
    display(Image.fromarray(run_grad_cam_on_image(model=model,
                                                  target_layer=target_layer,
                                                  targets_for_gradcam=targets_for_gradcam,
                                                  reshape_transform=None)))
    print_top_categories(model, img_tensor)

ValueError: too many values to unpack (expected 4)